In [1]:
# looking at what happens to the encoder when we inject a decoder vector.

In [2]:
import os
import sys
sys.path.append(os.path.abspath('..'))

import torch
from torch.utils.data import DataLoader
from transformer_lens import HookedTransformer
from transformer_lens import utils as tutils
from transformer_lens.evals import make_pile_data_loader, evaluate_on_dataset

from functools import partial
from datasets import load_dataset
from tqdm import tqdm

import einops

from sae_lens import SAE
# from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes
# from sae_lens import SparseAutoencoder, ActivationsStore

from steering.evals_utils import evaluate_completions, multi_criterion_evaluation
from steering.utils import normalise_decoder
from steering.patch import generate, scores_2d, patch_resid

# from sae_vis.data_config_classes import SaeVisConfig
# from sae_vis.data_storing_fns import SaeVisData

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

torch.set_grad_enabled(False)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HookedTransformer.from_pretrained("gemma-2b", device=device)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b into HookedTransformer


In [4]:
hp6 = "blocks.6.hook_resid_post"

sae6, _, _ = SAE.from_pretrained(
    release = "gemma-2b-res-jb", # see other options in sae_lens/pretrained_saes.yaml
    sae_id = hp6, # won't always be a hook point
    device = 'cpu'
)

sae6 = sae6.to(device)
normalise_decoder(sae6)

In [5]:
# intelligence = sae6.W_dec[10351]   # intelligence and genius
writing = sae6.W_dec[1058]  # writing
anger = sae6.W_dec[1062]  # anger
london = sae6.W_dec[10138]  # London
wedding = sae6.W_dec[8406]  # wedding
broad_wedding = sae6.W_dec[2378] # broad wedding

broad_poetry = sae6.W_dec[11067]  # broad poetry

In [ ]:
batch_size = 8
data = load_dataset("NeelNanda/c4-code-20k", split="train")
tokenized_data = tutils.tokenize_and_concatenate(data, model.tokenizer, max_length=32)
tokenized_data = tokenized_data.shuffle(42)
loader = DataLoader(tokenized_data, batch_size=batch_size)

In [44]:
steer = london
scale = 100

In [45]:
n_steps = 10

diffs = torch.zeros(sae6.cfg.d_sae, device=sae6.W_dec.device)

for batch_idx, batch in enumerate(loader):

    _, acts = model.run_with_cache(batch['tokens'], names_filter=hp6)
    acts = acts[hp6]
    acts = acts.reshape(-1, acts.shape[-1]) 

    pre_distribution = sae6.encode(acts)
    post_distribution = sae6.encode(acts + steer*scale)

    diff = post_distribution - pre_distribution
    diffs += diff.sum(dim=0)

    if batch_idx >= n_steps - 1:
        break

diffs /= (n_steps*32*8)

In [46]:
top_v, top_i = torch.topk(diffs, 10, dim=-1)
print(top_i)
print(top_v)


tensor([10138, 12090,  2675,  2151,  6946,  9104, 15631,  4963, 15173,   891],
       device='cuda:0')
tensor([91.7861,  3.9049,  2.7118,  2.1605,  1.7423,  1.6251,  1.6011,  1.5770,
         1.5612,  1.4861], device='cuda:0')


In [47]:
# bottom_v, bottom_i = torch.topk(-diffs, 10, dim=-1)
# print(bottom_i)
# print(bottom_v)

In [48]:
# now we try the same but just by cosine similarity to encoder vectors

In [49]:
normed_enc = sae6.W_enc / sae6.W_enc.norm(dim=0, keepdim=True)
enc_sims = (steer @ normed_enc) / steer.norm()


In [43]:
top_v, top_i = torch.topk(enc_sims, 10, dim=-1)
print(top_i)
print(top_v)

tensor([10138, 10195,  2675,  4846,  3206, 15631,  1921,  6946,  8115, 12090],
       device='cuda:0')
tensor([0.2728, 0.0448, 0.0447, 0.0446, 0.0444, 0.0438, 0.0428, 0.0422, 0.0418,
        0.0416], device='cuda:0')


In [27]:
# texts = generate(model, hooks=[(hp6, partial(patch_resid, steering=broad_poetry, scale=80))], prompt='I think', n_samples=32, batch_size=32)
# texts